In [195]:
using LinearAlgebra 
using Plots #Graph
using StaticArrays 
using SparseArrays  
using Optim
using Arpack  #Eigenvalues and eigenvectors
using LaTeXStrings #Titles and labels 
using JLD #Save data
#definition of N,M,D
global N=3
global M=3
global D=Int((factorial(M+N-1))/(factorial(M-1)*factorial(N)))  

#i-prime number function(suggested form) 
p(i)=100i+3 

#Generate a basis for N = M
function generate_basis(N, M)
    D = prod(max(N, M):N+M-1) ÷ prod(1:min(N, M))
    basis = [zeros(Int, M) for _ in 1:D]
    basis[1][1] = N
    for t = 2:D
        if basis[t-1][M] != 0
            k = M - 1
        else
            k = M
        end
        while k > 0 && basis[t-1][k] == 0
            k -= 1
        end
        @views basis[t][1:k-1] .= basis[t-1][1:k-1]
        basis[t][k] = basis[t-1][k] - 1
        basis[t][k+1] = N - sum(@view(basis[t][1:k])) 
        end 
    return basis  
end

generate_basis (generic function with 1 method)

In [2]:
v=generate_basis(N,M)

10-element Vector{Vector{Int64}}:
 [3, 0, 0]
 [2, 1, 0]
 [2, 0, 1]
 [1, 2, 0]
 [1, 1, 1]
 [1, 0, 2]
 [0, 3, 0]
 [0, 2, 1]
 [0, 1, 2]
 [0, 0, 3]

In [3]:
function occupation(i, v) 
     if(v[i] >= 1)  
        return  v[i]  
    else 
        return 0;
    end  
end 

occupation (generic function with 1 method)

In [4]:
function matrixoccupation(k, D) 
    mat = spzeros(D,D)
    for j in 1:D 
        mat[j,j] = occupation(k, v[j]) 
    end   
    return mat 
end

matrixoccupation (generic function with 1 method)

In [177]:
#Parity order parameter
Op =exp(1im*(pi)*Matrix(matrixoccupation(2,D)) - I) 
Op = real(Op)

10×10 Matrix{Float64}:
 0.367879   0.0       0.0       0.0       …  0.0        0.0       0.0
 0.0       -0.367879  0.0       0.0          0.0        0.0       0.0
 0.0        0.0       0.367879  0.0          0.0        0.0       0.0
 0.0        0.0       0.0       0.367879     0.0        0.0       0.0
 0.0        0.0       0.0       0.0          0.0        0.0       0.0
 0.0        0.0       0.0       0.0       …  0.0        0.0       0.0
 0.0        0.0       0.0       0.0          0.0        0.0       0.0
 0.0        0.0       0.0       0.0          0.367879   0.0       0.0
 0.0        0.0       0.0       0.0          0.0       -0.367879  0.0
 0.0        0.0       0.0       0.0          0.0        0.0       0.367879

In [210]:
#String Order parameter 
Os = (Matrix(matrixoccupation(2,D)) - I)*(exp(1im*pi*Matrix(matrixoccupation(2,D)) - I)*Matrix(matrixoccupation(3,D)) - I) 
Os = real(Os)

10×10 Matrix{Float64}:
 1.0  0.0  0.0        0.0  0.0  0.0        0.0   0.0       0.0   0.0
 0.0  0.0  0.0        0.0  0.0  0.0        0.0   0.0       0.0   0.0
 0.0  0.0  0.632121   0.0  0.0  0.0        0.0   0.0       0.0   0.0
 0.0  0.0  0.0       -1.0  0.0  0.0        0.0   0.0       0.0   0.0
 0.0  0.0  0.0        0.0  0.0  0.0        0.0   0.0       0.0   0.0
 0.0  0.0  0.0        0.0  0.0  0.264241   0.0   0.0       0.0   0.0
 0.0  0.0  0.0        0.0  0.0  0.0       -2.0   0.0       0.0   0.0
 0.0  0.0  0.0        0.0  0.0  0.0        0.0  -0.632121  0.0   0.0
 0.0  0.0  0.0        0.0  0.0  0.0        0.0   0.0       0.0   0.0
 0.0  0.0  0.0        0.0  0.0  0.0        0.0   0.0       0.0  -0.103638

In [212]:
#Optim Example for 
#f(x) = (1.0 - x[1])^2 + 100.0 * (x[2] - x[1]^2)^2  
#x0 = [0.0, 0.0]  
#optimize(f, x0)  
# optimize(f, zeros(2) )  
##optimize(x->dot(x, Os*x), zeros(D)) ###NO CONSTRAINTS! so we will use JuMP package
using JuMP
import Ipopt

In [213]:

function ParityOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, x[1:D] >= 0)
    @objective(model, Max, z) 
    @constraint(model, dot(x, Op*x) == z)
    @constraint(model, sum(x.^2) == 1)
    optimize!(model)
    if verbose
        print(model)
        println("Objective value: ", objective_value(model))
        println("Op = ", value(z))
        for i in 1:D 
            println("x_$i = ", value(x[i]) ) 
        end
    end
    return
end

ParityOrder()

Objective value: 0.36787943935442524
Op = 0.36787943935442524
x_1 = 0.408248289771318
x_2 = 3.957948802575254e-5
x_3 = 0.4082482899917525
x_4 = 0.4082482899917525
x_5 = 3.957948802575254e-5
x_6 = 0.4082482899917525
x_7 = 3.957948802575255e-5
x_8 = 0.40824829002024177
x_9 = 3.957948802575254e-5
x_10 = 0.4082482899917525


In [211]:
function StringOrder(; verbose = true)
    model = Model(Ipopt.Optimizer)
    set_silent(model)
    @variable(model, z)
    @variable(model, x[1:D] >= 0)
    @objective(model, Max, z) 
    @constraint(model, dot(x, Op*x) == z)
    @constraint(model, sum(x.^2) == 1)
    optimize!(model)
    if verbose
        print(model)
        println("Objective value: ", objective_value(model))
        println("Os = ", value(z))
        for i in 1:D 
            println("x_$i = ", value(x[i]) ) 
        end
    end
    return
end

StringOrder()

Objective value: 0.36787943935442524
Op = 0.36787943935442524
x_1 = 0.408248289771318
x_2 = 3.957948802575254e-5
x_3 = 0.4082482899917525
x_4 = 0.4082482899917525
x_5 = 3.957948802575254e-5
x_6 = 0.4082482899917525
x_7 = 3.957948802575255e-5
x_8 = 0.40824829002024177
x_9 = 3.957948802575254e-5
x_10 = 0.4082482899917525
